In [ ]:
import NBInclude: @nbinclude
@nbinclude("RecommendationsBase.ipynb");

## Curated TV Series

In [ ]:
first(
    rec_df |>
    tv_display |>
    implicit_display |>
    explicit_display |>
    unrelated_display |>
    unseen_display,
    50,
)

## Popular items

In [ ]:
first(
    rec_df |> tv_display |> inv_implicit_display |> unrelated_display |> unseen_display,
    10,
)

## Highly Rated Items

In [ ]:
first(
    rec_df |> tv_display |> inv_explicit_display |> unrelated_display |> unseen_display,
    10,
)

## Curated Specials

In [ ]:
first(
    rec_df |>
    inv_tv_display |>
    implicit_display |>
    explicit_display |>
    unrelated_display |>
    unseen_display,
    10,
)

## Continue Watching TV Series

In [ ]:
first(rec_df |> tv_display |> related_display |> unseen_display, 10)

## Continue Watching Specials

In [ ]:
first(rec_df |> inv_tv_display |> related_display |> unseen_display, 10)